### **This code automatically collect the Landsat images from 1986-2020.**

In [ ]:
## Mount on google drive
from google.colab import drive
drive.mount('/content/drive/')
## Authenticate to Google Cloud
from google.colab import auth
auth.authenticate_user()
# Authenticate to Earth Engine
import ee
ee.Authenticate()
ee.Initialize()


In [10]:
import time
import os


In [11]:
## Super-parameter
####################################################
## Region for prediction
Area_sz = ee.FeatureCollection('users/xin_luo/Impervious_Mapping_SZ/shenzhen_shp')
Area_sz_bound = Area_sz.geometry().bounds()
Area_sz_bound_expand = Area_sz_bound.buffer(10000).bounds()

## Data features
data_source_l5 = 'LANDSAT/LT05/C01/T1_SR'
data_source_l7 = "LANDSAT/LE07/C01/T1_SR"
data_source_l8 = 'LANDSAT/LC08/C01/T1_SR'
Bands_l5 = Bands_l7 = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
Bands_l8 = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']  # determine the bands to be collected
## Patch size: Kernel_size + Kernel_buffer
Kernel_size = [208, 208]
Kernel_buffer = [48, 48]   # left buffer + right buffer = 48, the same to up and down
Buffered_shape = [Kernel_size[0]+Kernel_buffer[0], Kernel_size[1]+Kernel_buffer[1]]

## Output path of Google Drive 
Image_Folder = 'EE_Image'   # !can't directly write into the second-level directory


In [12]:
date_years = ['1986','1987','1988','1989','1990','1991','1992','1993','1994',\
              '1995','1996','1997','1998','1999','2000','2001','2002','2003',\
              '2004','2005','2006','2007','2008','2009','2010','2011','2012',\
              '2013','2014','2015','2016','2017','2018','2019','2020']
date_years_l5 = ['1986','1987','1988','1989','1990','1991','1992','1993',\
                 '1994','1995','1996','1997','1998','1999','2000','2001',\
                 '2002','2003','2004','2005','2006','2007','2008','2009',\
                 '2010','2011']
date_years_l7 = ['2012']
date_years_l8 = ['2013','2014','2015','2016','2017','2018','2019','2020']

def getDataParameter(date_year):
    '''
    input: 
        the date(year) of the landsat image
    ouput:
        the data source (l5, l7, or l8) and bands selection 
    '''
    if date_year in date_years_l5:
        data_source = data_source_l5
        Bands = Bands_l5
    elif date_year in date_years_l7:
        data_source = data_source_l7
        Bands = Bands_l7
    elif date_year in date_years_l8:
        data_source = data_source_l8
        Bands = Bands_l8
    return data_source, Bands

In [16]:
for date_year in date_years:
    print(date_year)
    ## obain the image date and image source
    date_start = date_year + '-01-01'  # date of collected data
    date_end = date_year + '-12-31'
    data_source, Bands = getDataParameter(date_year)
    ## define the export name
    Image_Export_Name = 'Landsat_SZ_' + date_year

    ## 1. image collection and image preprocessing.
    ################################################### 
    image_collection = ee.ImageCollection(data_source)\
                        .filterDate(date_start, date_end)\
                        .filterBounds(Area_sz)
    ## 1) cloud mask
    def cloudMask(image):
        qa = image.select('pixel_qa')
        mask = qa.bitwiseAnd(1<<5).eq(0)
        return image.updateMask(mask)
    image_mask = image_collection.map(cloudMask).median()

    ## 2) fill the null value of the cloud-masked image.
    nullMask = image_mask.mask().Not()
    img_nullMask = image_collection.map(lambda image_mask: image_mask.updateMask(nullMask)).median()
    img_fill = image_mask.unmask(0)\
            .add(img_nullMask.unmask(0))\
            .clip(Area_sz_bound_expand)\
            .divide(10000)\
            .select(Bands)

    ## 1. Export the collected image
    ##################################################
    # Specify patch and file dimensions.
    image_export_options = {
            'patchDimensions': Kernel_size,    #note: this size not contains buffered region
            'maxFileSize': 504857600,   #  Memory for per TFRecord file
            'kernelSize': Kernel_buffer,  ## this parameter is required for the prediction data preparation.
            'compressed': True
            }

    # Setup the task.
    image_task = ee.batch.Export.image.toDrive(
            image = img_fill,
            description = Image_Export_Name,
            fileNamePrefix = Image_Export_Name,
            folder = Image_Folder,
            scale=30,
            region = Area_sz_bound_expand,
            fileFormat='TFRecord',
            formatOptions=image_export_options,  # Corresponding to the TFRecord export
            )

    # Start the task.
    image_task.start()

while image_task.active():
    print('Polling for task (id: {}).'.format(image_task.id))
    time.sleep(30)
print('Done with image export.')


2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Polling for task (id: 3ZEW2XBVVQG4226KL2ATOKLJ).
Polling for task (id: 3ZEW2XBVVQG4226KL2ATOKLJ).
Polling for task (id: 3ZEW2XBVVQG4226KL2ATOKLJ).
Polling for task (id: 3ZEW2XBVVQG4226KL2ATOKLJ).
Polling for task (id: 3ZEW2XBVVQG4226KL2ATOKLJ).
Polling for task (id: 3ZEW2XBVVQG4226KL2ATOKLJ).
Polling for task (id: 3ZEW2XBVVQG4226KL2ATOKLJ).
Polling for task (id: 3ZEW2XBVVQG4226KL2ATOKLJ).
Polling for task (id: 3ZEW2XBVVQG4226KL2ATOKLJ).
Polling for task (id: 3ZEW2XBVVQG4226KL2ATOKLJ).
Polling for task (id: 3ZEW2XBVVQG4226KL2ATOKLJ).
Polling for task (id: 3ZEW2XBVVQG4226KL2ATOKLJ).
Polling for task (id: 3ZEW2XBVVQG4226KL2ATOKLJ).
Polling for task (id: 3ZEW2XBVVQG4226KL2ATOKLJ).
Polling for task (id: 3ZEW2XBVVQG4226KL2ATOKLJ).
Polling for task (id: 3ZEW2XBVVQG4226KL2ATOKLJ).
Polling for task (id: 3ZEW2XBVVQG4226KL2ATOKLJ).
Polling for task (id: 3ZEW2XBVVQG4226KL2ATOKLJ).
Polling for task (id: 3ZEW2XBVVQG4226KL2ATOKLJ).

In [17]:
############### Move the file to the target directory
## Get a list of all the files in the tfrecord data folder.
os.chdir('/content/drive/My Drive/EE_Image/')
filesList = !ls -1
PredFilesList = [s for s in filesList \
                 if '.tfrecord.gz' in s or '.json' in s]

for file in PredFilesList:
    !mv {file} "Landsat_image_sz/"
